<a href="https://colab.research.google.com/github/kwondogeun/green/blob/main/bgquery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 구글 연동 **예제**

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


# GCP Project 연동
- 빅쿼리 클라이언트에 접속

In [2]:
project_id = "bq-sampleabde"

In [3]:
from google.cloud import bigquery

client = bigquery.Client(project=project_id)

df = client.query('''
  SELECT * FROM `basketball.mascots`
''').to_dataframe()

df

,id,market,name,mascot,mascot_name,mascot_common_name,tax_subspecies,tax_species,tax_genus,tax_family,tax_order,tax_class,tax_phylum,tax_kingdom,tax_domain,non_tax_type,flag
0,2959bd24-7007-41ae-a3a3-abdf26888cfc,Tulsa,Golden Hurricane,Hurricane,Captain Cane,None,None,None,None,None,None,None,None,None,None,Weather,1
1,ad4bc983-8d2e-4e6f-a8f9-80840a786c64,Arizona State,Sun Devils,Devil,Sparky,None,None,None,None,None,None,None,None,None,None,Devils,1
2,f2d01b77-0f5d-4574-9e49-2a3eaf822e44,Drexel,Dragons,Dragon,Mario the Magnificent,None,None,None,None,None,None,None,None,None,None,Dragons,1
3,b47d10b8-a2a5-47df-a2f9-7bd0b9d51beb,Bradley,Braves,Gargoyle,Kaboom!,None,None,None,None,None,None,None,None,None,None,None,1
4,0113eea0-c943-4fff-9780-ae0fb099e7ef,Canisius,Golden Griffins,Griffin,Petey,None,None,None,None,None,None,None,None,None,None,Griffins,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,683ab61f-546f-44da-b085-c3a5740554aa,Stanford,Cardinal,California Redwood,Stanford Tree,California Redwood,None,sempervirens,Sequoia,Cupressaceae,Pinales,Pinopsida,Pinophyta,Plantae,Eukaryota,None,1
347,f38b58de-3273-4723-a78b-2f918e9aa2d5,Wichita State,Shockers,Wheat,WuShock,Wheat,None,None,Triticum,Poaceae,Cyperales,Liliopsida,Magnoliophyta,Plantae,Eukaryota,None,1
348,63cad858-8654-4956-8438-8fe79cd1208a,Louisiana-Lafayette,Ragin' Cajuns,Cayenne Pepper,Cayenne,Cayenne,None,annuum,Capsicum,Solanaceae,Solanales,Magnoliopsida,Tracheophyta,Plantae,Eukaryota,None,1
349,8cd24de1-2be8-4b77-bcfa-47be0495a5f4,Syracuse,Orange,Orange,Otto,Orange,None,None,Citrus,Rutaceae,Sapindales,Magnoliopsida,Magnoliophyta,Plantae,Eukaryota,None,1


# 서브쿼리 테스트


In [4]:
df = client.query('''
  WITH matchups AS (
  SELECT g.win_team_id
  , g.lose_team_id
  , (SELECT win_masc.tax_family FROM `basketball.mascots` win_masc WHERE win_masc.id = g.win_team_id) AS tax_family_winner
  , (SELECT win_masc.tax_genus FROM `basketball.mascots` win_masc WHERE win_masc.id = g.win_team_id) AS tax_genus_winner
  , (SELECT lose_masc.tax_family FROM `basketball.mascots` lose_masc WHERE lose_masc.id = g.lose_team_id) AS tax_family_loser
  , (SELECT lose_masc.tax_genus FROM `basketball.mascots` lose_masc WHERE lose_masc.id = g.lose_team_id) AS tax_genus_loser
  FROM `bigquery-public-data.ncaa_basketball.mbb_historical_tournament_games` g
  )
  SELECT
    SUM(IF(tax_family_winner = "Felidae" AND tax_genus_loser = "Canis", 1, 0)) AS num_cat_wins,
    SUM(IF(tax_genus_winner = "Canis" AND tax_family_loser = "Felidae", 1, 0)) AS num_dog_wins
  FROM matchups
''').to_dataframe()

df

,num_cat_wins,num_dog_wins
0,34,43


# 데이터 추가

In [5]:
import pandas as pd 
df = pd.read_csv("/content/sample_data/california_housing_train.csv")
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


# 테이블 추가하기

In [9]:
# Dataset Project Name
TABLE_NAME = 'basketball.housting_train'
project_id = 'bq-sampleabde'

df.to_gbq(TABLE_NAME, project_id, if_exists='append')

1it [00:06,  6.21s/it]
